In [ ]:
# !pip install sqlalchemy pyodbc pandas numpy torch matplotlib pillow azure-storage-blob azure-identity
# !pip install pymssql torchvision scikit-learn
# !pip uninstall -y pyarrow
# !pip install pyarrow
# !pip install fastparquet  


In [32]:
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine, text
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
import io
import matplotlib.pyplot as plt
import pymssql
from sklearn.preprocessing import StandardScaler
import os
import pickle
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.identity import DefaultAzureCredential
import pyarrow
import fastparquet


## 2. Configuration


In [ ]:
# Database Configuration
SERVER = "server"
DATABASE = "db"
USERNAME = "username"
PASSWORD = "password"

STORAGE_ACCOUNT_NAME = "account"  
STORAGE_ACCOUNT_KEY = "key+ASt7cDLtQ=="  

CONTAINER_NAME_IMAGES = "container"  # Container name for CT scan images
CONTAINER_NAME_DATA = "container"    # Container name for data files (parquet)

PARQUET_BLOB_PATH = "city_wellness_curated.parquet"  
MEDICAL_IMAGES_PREFIX = "medical_images/"  

MODEL_DIR = 'Database/models'

def get_connection():
    return pymssql.connect(
        server=SERVER,
        user=USERNAME,
        password=PASSWORD,
        database=DATABASE,
        port=1433,
        tds_version="7.4"
    )

def get_blob_service_client():
    try:
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={STORAGE_ACCOUNT_KEY};EndpointSuffix=core.windows.net"
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        return blob_service_client
    except Exception as e:
        print(f"Warning: Could not initialize blob service client: {e}")
        print("Make sure to update STORAGE_ACCOUNT_NAME and STORAGE_ACCOUNT_KEY")
        return None

blob_service_client = get_blob_service_client()


 Database connection ready
 Azure Blob Storage client ready


## 3. Database Helper Functions


In [34]:
def get_customer(customer_id):
    conn = get_connection()
    cursor = conn.cursor(as_dict=True)
    cursor.execute("SELECT * FROM Customer WHERE CustomerID = %s", (customer_id,))
    row = cursor.fetchone()
    conn.close()
    return row

def get_health_factors(customer_id):
    conn = get_connection()
    cursor = conn.cursor(as_dict=True)
    cursor.execute("""
        SELECT FactorName, FactorValue
        FROM CustomerHealthFactor
        WHERE CustomerID = %s
    """, (customer_id,))
    rows = cursor.fetchall()
    conn.close()
    return rows

def insert_health_factor(customer_id, factor_name, factor_value, year=2024, asset_id=1):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO CustomerHealthFactor (CustomerID, FactorName, FactorValue, FactorYear, SourceAssetID)
        VALUES (%s, %s, %s, %s, %s)
    """, (customer_id, factor_name, factor_value, year, asset_id))
    conn.commit()
    conn.close()

def get_customer_image_blob(customer_id):
    conn = get_connection()
    cursor = conn.cursor(as_dict=True)
    cursor.execute("""
        SELECT TOP 1 ImageFileName 
        FROM CustomerHealthFactor 
        WHERE CustomerID = %s AND ImageFileName IS NOT NULL
        ORDER BY FactorID DESC
    """, (customer_id,))
    row = cursor.fetchone()
    conn.close()
    return row["ImageFileName"] if row and "ImageFileName" in row else None

def insert_contract(customer_id):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO Contract (AccountID, CompanyCode, OwnerCustomerID, PayerCustomerID,
                              PolicyNumber, IssueDate, Status)
        VALUES (1, 'NY', %s, %s, CONCAT('POL', FLOOR(RAND()*1000000)), GETDATE(), 'Pending')
    """, (customer_id, customer_id))
    conn.commit()
    cursor.execute("SELECT @@IDENTITY AS ContractID")
    contract_id = cursor.fetchone()[0]
    conn.close()
    return contract_id

def insert_contract_benefit(contract_id, benefit_type="Health", coverage_amount=50000.00):
    conn = get_connection()
    cursor = conn.cursor()
    try:
        # Based on actual schema: ContractBenefitID, ContractID, BenefitType, CoverageAmount, EffectiveDate, EndDate
        cursor.execute("""
            INSERT INTO ContractBenefit (ContractID, BenefitType, CoverageAmount, EffectiveDate, EndDate)
            VALUES (%s, %s, %s, GETDATE(), DATEADD(YEAR, 2, GETDATE()))
        """, (contract_id, benefit_type, coverage_amount))
        conn.commit()
        cursor.execute("SELECT @@IDENTITY AS ContractBenefitID")
        contract_benefit_id = cursor.fetchone()[0]
        conn.close()
        return contract_benefit_id
    except Exception as e:
        conn.close()
        raise ValueError(f"Could not insert ContractBenefit: {e}")

def insert_contract_premium(contract_benefit_id, premium):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO ContractPremium (ContractBenefitID, WritingAssociateID, PremiumAmount,
                                     Frequency, EffectiveDate)
        VALUES (%s, 1, %s, 'Monthly', GETDATE())
    """, (contract_benefit_id, premium))
    conn.commit()
    conn.close()

def refresh_materialized_views():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("EXEC Refresh_Materialized_Views")
    conn.commit()
    conn.close()

print(" Database helper functions ready")


✓ Database helper functions ready


## 4. Load ML Models


In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

#  ResNet18
ct_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def load_ct_model(model_path):
    model = models.resnet18(weights="IMAGENET1K_V1")
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 1),
        nn.Sigmoid()
    )
    
    if model_path and os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Loaded CT model from {model_path}")
    else:
        print("Warning: Using ImageNet-initialized CT model. Train and save model first.")
    
    model = model.to(device)
    model.eval()
    return model

# Load CT model
ct_model = load_ct_model(os.path.join(MODEL_DIR, 'ct_risk_model.pth'))


Using device: cpu
 Loaded CT model from /Users/grishmadeshmukh/Desktop/NYUMasters/Sem3/Database/part4/models/ct_risk_model.pth


In [36]:
class WellnessNN(nn.Module):
    def __init__(self, input_size=9):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x)

def load_wellness_model(model_path, scaler_path, input_size=9):
    model = WellnessNN(input_size=input_size)
    
    if model_path and os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Loaded wellness model from {model_path}")
    else:
        print("Warning: Using untrained wellness model. Train and save model first.")
    
    model = model.to(device)
    model.eval()
    
    # Load scaler
    scaler = None
    if scaler_path and os.path.exists(scaler_path):
        with open(scaler_path, 'rb') as f:
            scaler = pickle.load(f)
        print(f"Loaded scaler from {scaler_path}")
    else:
        print("Warning: No scaler found. Using StandardScaler (will need fitting).")
        scaler = StandardScaler()
    
    return model, scaler

wellness_model, wellness_scaler = load_wellness_model(
    model_path=os.path.join(MODEL_DIR, 'wellness_model.pth'),
    scaler_path=os.path.join(MODEL_DIR, 'wellness_scaler.pkl')
)


 Loaded wellness model from /Users/grishmadeshmukh/Desktop/NYUMasters/Sem3/Database/part4/models/wellness_model.pth
 Loaded scaler from /Users/grishmadeshmukh/Desktop/NYUMasters/Sem3/Database/part4/models/wellness_scaler.pkl


## 5. Azure Blob Storage Functions


In [ ]:
# Cache for city wellness data 
_city_wellness_cache = None

def load_image_from_blob(blob_name, container_name=CONTAINER_NAME_IMAGES):
    if blob_service_client is None:
        raise ValueError("Azure Blob Storage client not initialized. Check configuration.")
    
    if not blob_name.startswith("medical_images/"):
        blob_name = MEDICAL_IMAGES_PREFIX + blob_name
    
    try:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        blob_data = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_data)).convert("RGB")
        return img
    except Exception as e:
        raise FileNotFoundError(f"Could not load image '{blob_name}' from blob storage: {e}")

def load_city_wellness_from_parquet(parquet_blob_path=None, container_name=CONTAINER_NAME_DATA):
    global _city_wellness_cache
    
    # Use cache if already loaded
    if _city_wellness_cache is not None:
        return _city_wellness_cache
    
    if blob_service_client is None:
        raise ValueError("Azure Blob Storage client not initialized. Check configuration.")
    
    if parquet_blob_path is None:
        parquet_blob_path = PARQUET_BLOB_PATH
    
    try:
        # Download parquet file from blob storage
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=parquet_blob_path)
        blob_data = blob_client.download_blob().readall()
        
        try:
            df = pd.read_parquet(io.BytesIO(blob_data), engine='pyarrow')
        except (ValueError, ImportError, AttributeError) as e:
            if 'pandas.period' in str(e) or 'already defined' in str(e):
                print("  pyarrow conflict detected, trying fastparquet...")
                try:
                    df = pd.read_parquet(io.BytesIO(blob_data), engine='fastparquet')
                except ImportError:
                    raise ImportError("fastparquet not installed. Please run: !pip install fastparquet")
            else:
                try:
                    df = pd.read_parquet(io.BytesIO(blob_data), engine='fastparquet')
                except ImportError:
                    raise ImportError("Neither pyarrow nor fastparquet working. Please reinstall: !pip uninstall -y pyarrow && !pip install pyarrow")
        
        if "Obesity levels(Country)" in df.columns:
            df["Obesity levels(Country)"] = df["Obesity levels(Country)"].astype(str).str.replace("%", "", regex=False)
        if "Cost of a bottle of water(City)" in df.columns:
            df["Cost of a bottle of water(City)"] = df["Cost of a bottle of water(City)"].astype(str).str.replace("£", "").str.replace("$", "").str.replace("€", "")
        if "Cost of a monthly gym membership(City)" in df.columns:
            df["Cost of a monthly gym membership(City)"] = df["Cost of a monthly gym membership(City)"].astype(str).str.replace("£", "").str.replace("$", "").str.replace("€", "")
        
        numeric_cols = [
            "Sunshine hours(City)",
            "Obesity levels(Country)",
            "Life expectancy(years) (Country)",
            "Pollution(Index score) (City)",
            "Annual avg. hours worked",
            "Happiness levels(Country)",
            "Outdoor activities(City)",
            "Cost of a bottle of water(City)",
            "Cost of a monthly gym membership(City)"
        ]
        
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        for col in numeric_cols:
            if col in df.columns:
                df[col] = df[col].fillna(df[col].mean())
        
        # Cache the dataframe
        _city_wellness_cache = df
        print(f" Loaded city wellness data from blob: {parquet_blob_path}")
        return df
        
    except Exception as e:
        raise FileNotFoundError(f"Could not load city wellness parquet '{parquet_blob_path}' from blob storage: {e}")

def load_city_features(city, parquet_blob_path=None):
    """Load city wellness features from Azure Blob Storage (parquet)."""
    df = load_city_wellness_from_parquet(parquet_blob_path)
    
    city_row = df[df['City'].str.lower() == city.lower()]
    if city_row.empty:
        raise ValueError(f"City '{city}' not found in city wellness data")
    
    numeric_cols = [
        "Sunshine hours(City)",
        "Obesity levels(Country)",
        "Life expectancy(years) (Country)",
        "Pollution(Index score) (City)",
        "Annual avg. hours worked",
        "Happiness levels(Country)",
        "Outdoor activities(City)",
        "Cost of a bottle of water(City)",
        "Cost of a monthly gym membership(City)"
    ]
    
    features = []
    for col in numeric_cols:
        if col in city_row.columns:
            val = city_row[col].iloc[0]
            features.append(float(val) if pd.notna(val) else 0.0)
        else:
            raise ValueError(f"Column '{col}' not found in city wellness data")
    
    return np.array(features, dtype=float)

print(" Azure Blob Storage functions ready")


 Azure Blob Storage functions ready


## 6. ML Score Computation Functions


In [ ]:
def compute_ct_risk(image_source, from_blob=True):
    if from_blob:
        img = load_image_from_blob(image_source)
    else:
        if not os.path.exists(image_source):
            raise FileNotFoundError(f"Image not found: {image_source}")
        img = Image.open(image_source).convert("RGB")
    
    img_tensor = ct_transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        score = ct_model(img_tensor).item()
    
    return float(score)

def update_ct_risk_score(customer_id, image_source, from_blob=True):
    score = compute_ct_risk(image_source, from_blob=from_blob)
    insert_health_factor(customer_id, "CT_RiskScore", score)
    return score

def update_wellness_score(customer_id, city):
    features = load_city_features(city)
    
    # Scale features using the same scaler from training
    if hasattr(wellness_scaler, 'mean_') and wellness_scaler.mean_ is not None:
        features_scaled = wellness_scaler.transform(features.reshape(1, -1))
    else:
        print("Warning: Scaler not fitted. Using raw features.")
        features_scaled = features.reshape(1, -1)
    
    features_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)

    with torch.no_grad():
        score = float(wellness_model(features_tensor).item())

    insert_health_factor(customer_id, "WellnessScore", score)
    return score

def ensure_scores(customer_id, city=None, ct_image_blob=None, from_blob=True):
    existing = get_health_factors(customer_id)
    scores = {"CT_RiskScore": None, "WellnessScore": None}

    for row in existing:
        if row["FactorName"] in scores:
            scores[row["FactorName"]] = row["FactorValue"]

    # Get city from customer if not provided
    if city is None:
        customer = get_customer(customer_id)
        if customer and "City" in customer:
            city = customer["City"]
        else:
            raise ValueError(f"City not provided and not found in customer record for ID {customer_id}")

    # Get CT image blob from database if not provided
    if scores["CT_RiskScore"] is None:
        if ct_image_blob is None:
            ct_image_blob = get_customer_image_blob(customer_id)
            if ct_image_blob is None:
                raise ValueError(f"CT image blob not found for customer {customer_id}. Provide ct_image_blob parameter.")
        
        scores["CT_RiskScore"] = update_ct_risk_score(customer_id, ct_image_blob, from_blob=from_blob)

    if scores["WellnessScore"] is None:
        scores["WellnessScore"] = update_wellness_score(customer_id, city)

    return scores

def compute_premium(base, ct_score, wellness_score):
    """Calculate insurance premium based on ML scores."""
    α = 0.4  # CT risk weight
    β = 0.6  # Wellness weight
    premium = base * (1 + α * ct_score + β * (1 - wellness_score))
    return float(premium)

print(" ML score computation functions ready")


 ML score computation functions ready


## 7. Helper Functions - List Available Data


In [ ]:
def list_available_cities_from_blob(parquet_blob_path=None):
    """List all available cities from the parquet file in blob storage."""
    try:
        df = load_city_wellness_from_parquet(parquet_blob_path)
        if 'City' in df.columns:
            available_cities = df['City'].tolist()
            print(f"Found {len(available_cities)} cities in blob storage:")
            print("\nAvailable cities:")
            for i, city in enumerate(available_cities, 1):
                print(f"  {i}. {city}")
            print(f"\nUse any of these city names in the workflow below")
            print(f"Note: City matching is case-insensitive")
            return available_cities
        else:
            print("Warning: 'City' column not found in parquet file")
            return []
    except Exception as e:
        print(f"Could not load cities from blob storage: {e}")
        print("Make sure Azure Blob Storage is configured correctly")
        return []

def list_available_ct_images_from_blob(container_name=CONTAINER_NAME_IMAGES, prefix=MEDICAL_IMAGES_PREFIX):
    if blob_service_client is None:
        print("Azure Blob Storage client not initialized. Cannot list images.")
        return []
    
    try:
        container_client = blob_service_client.get_container_client(container_name)
        blobs = container_client.list_blobs(name_starts_with=prefix)
        
        image_extensions = ('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')
        image_blobs = [blob.name for blob in blobs if blob.name.lower().endswith(image_extensions)]
        
        print(f"Found {len(image_blobs)} CT images in blob container '{container_name}' (prefix: '{prefix}'):")
        if image_blobs:
            print("\nFirst 10 available image blobs:")
            for i, blob_name in enumerate(image_blobs[:10], 1):
                print(f"  {i}. {blob_name}")
            if len(image_blobs) > 10:
                print(f"  ... and {len(image_blobs) - 10} more")
            print(f"\nUse any of these blob names in the workflow below")
            print(f"Note: You can use just the filename (e.g., 'covid_2020.02.25.20021568-p23-108%9.png')")
            print(f"      or the full path (e.g., 'medical_images/covid_2020.02.25.20021568-p23-108%9.png')")
        return image_blobs
    except Exception as e:
        print(f"Could not list images from blob storage: {e}")
        print("Make sure Azure Blob Storage is configured correctly")
        return []

# Uncomment to list available data:
# print("=== Available Cities ===")
# available_cities = list_available_cities_from_blob()
# print("\n=== Available CT Images ===")
# available_ct_images = list_available_ct_images_from_blob()


## 8. End-to-End Quote Generation Workflow

**Update the configuration below and run this cell to generate an insurance quote:**


## 9. User-Friendly Quote Generator

**Simple interface to generate insurance quotes. Just provide the inputs below:**


In [ ]:
# ============================================================================
# USER INPUT 
# ============================================================================
CUSTOMER_ID = 10          # Customer ID from database
BASE_PREMIUM = 200       # Base premium amount in USD
CITY = None              # Optional: City name (will auto-retrieve from customer if None)
CT_IMAGE_BLOB = None     # Optional: CT image blob name (will auto-retrieve from database if None)

# ============================================================================
# GENERATE QUOTE
# ============================================================================

try:
    print("=" * 70)
    print(" " * 20 + "INSURANCE QUOTE GENERATOR")
    print("=" * 70)
    print()
    
    # Get customer information
    customer = get_customer(CUSTOMER_ID)
    if not customer:
        raise ValueError(f" Customer {CUSTOMER_ID} not found in database")
    
    customer_name = f"{customer.get('FirstName', 'N/A')} {customer.get('LastName', 'N/A')}"
    customer_city = CITY or customer.get('City', 'N/A')
    
    print(f" Customer: {customer_name} (ID: {CUSTOMER_ID})")
    print(f" Location: {customer_city}")
    print()
    print("-" * 70)
    
    # Compute ML scores
    print("🔬 Computing Health Risk Assessment...")
    scores = ensure_scores(
        customer_id=CUSTOMER_ID,
        city=CITY,
        ct_image_blob=CT_IMAGE_BLOB,
        from_blob=True
    )
    
    ct_score = scores['CT_RiskScore']
    wellness_score = scores['WellnessScore']
    
    print(f"   CT Scan Risk Score:     {ct_score:.4f} ({ct_score*100:.1f}% risk)")
    print(f"   City Wellness Score:    {wellness_score:.4f} ({wellness_score*100:.1f}% wellness)")
    print()
    print("-" * 70)
    
    # Calculate premium
    print(" Calculating Premium...")
    final_premium = compute_premium(BASE_PREMIUM, ct_score, wellness_score)
    
    # Calculate adjustments
    ct_adjustment = BASE_PREMIUM * 0.4 * ct_score
    wellness_adjustment = BASE_PREMIUM * 0.6 * (1 - wellness_score)
    total_adjustment = ct_adjustment + wellness_adjustment
    adjustment_percent = (total_adjustment / BASE_PREMIUM) * 100
    
    print(f"   Base Premium:             ${BASE_PREMIUM:.2f}")
    print(f"   + CT Risk Adjustment:     +${ct_adjustment:.2f} ({0.4 * ct_score * 100:.1f}%)")
    print(f"   + Wellness Adjustment:    +${wellness_adjustment:.2f} ({0.6 * (1 - wellness_score) * 100:.1f}%)")
    print(f"   ─────────────────────────────────────────────")
    print(f"   Final Premium:            ${final_premium:.2f} ({adjustment_percent:+.1f}% change)")
    print()
    print("-" * 70)
    
    # Create contract
    print(" Creating Contract...")
    contract_id = insert_contract(CUSTOMER_ID)
    
    contract_benefit_id = None
    try:
        contract_benefit_id = insert_contract_benefit(contract_id, benefit_type="Health", coverage_amount=50000.00)
        insert_contract_premium(contract_benefit_id, final_premium)
        print(f"   ✓ Contract Created:        ID {contract_id}")
        print(f"   ✓ Benefit Added:           ID {contract_benefit_id} (Health, $50,000 coverage)")
        print(f"   ✓ Premium Recorded:         ${final_premium:.2f}/month")
    except ValueError as e:
        print(f" Contract created (ID: {contract_id}) but benefit/premium insertion failed")
        print(f" Error: {str(e)[:100]}")
    
    refresh_materialized_views()
    print()
    print("=" * 70)
    print(" " * 25 + "QUOTE SUMMARY")
    print("=" * 70)
    print()
    print(f"Customer:              {customer_name}")
    print(f"Customer ID:           {CUSTOMER_ID}")
    print(f"Location:              {customer_city}")
    print()
    print(f"Health Assessment:")
    print(f"  • CT Risk Score:      {ct_score:.4f} ({ct_score*100:.1f}% risk detected)")
    print(f"  • Wellness Score:     {wellness_score:.4f} ({wellness_score*100:.1f}% wellness)")
    print()
    print(f"Premium Breakdown:")
    print(f"  • Base Premium:       ${BASE_PREMIUM:.2f}")
    print(f"  • Risk Adjustments:   +${total_adjustment:.2f} ({adjustment_percent:+.1f}%)")
    print(f"  • Final Premium:      ${final_premium:.2f}/month")
    print()
    print(f"Contract Details:")
    print(f"  • Contract ID:        {contract_id}")
    if contract_benefit_id:
        print(f"  • Benefit ID:          {contract_benefit_id}")
        print(f"  • Coverage:            $50,000 (Health)")
    print()
    print("=" * 70)
    print(" " * 20 + "✓ Quote Generated Successfully!")
    print("=" * 70)
    
except Exception as e:
    print()
    print("=" * 70)
    print(" " * 25 + "! ERROR")
    print("=" * 70)
    print(f"\n{str(e)}\n")
    import traceback
    traceback.print_exc()


                    INSURANCE QUOTE GENERATOR

 Customer: James Walker (ID: 10)
 Location: New York

----------------------------------------------------------------------
🔬 Computing Health Risk Assessment...
   CT Scan Risk Score:     0.7273 (72.7% risk)
   City Wellness Score:    0.5549 (55.5% wellness)

----------------------------------------------------------------------
 Calculating Premium...
   Base Premium:             $200.00
   + CT Risk Adjustment:     +$58.18 (29.1%)
   + Wellness Adjustment:    +$53.41 (26.7%)
   ─────────────────────────────────────────────
   Final Premium:            $311.59 (+55.8% change)

----------------------------------------------------------------------
 Creating Contract...
   ✓ Contract Created:        ID 25
   ✓ Benefit Added:           ID 23 (Health, $50,000 coverage)
   ✓ Premium Recorded:         $311.59/month

                         QUOTE SUMMARY

Customer:              James Walker
Customer ID:           10
Location:              New

## 10. Model Retraining Module

**This module enables automatic retraining of ML models when new unstructured data is detected in Azure Blob Storage.**


In [ ]:
import json
from datetime import datetime

MODEL_VERSION_FILE = os.path.join(MODEL_DIR, 'model_versions.json')

def get_model_versions():
    if os.path.exists(MODEL_VERSION_FILE):
        with open(MODEL_VERSION_FILE, 'r') as f:
            return json.load(f)
    return {
        'ct_model': {'last_trained': None, 'data_hash': None},
        'wellness_model': {'last_trained': None, 'data_hash': None}
    }

def save_model_versions(versions):
    os.makedirs(MODEL_DIR, exist_ok=True)
    with open(MODEL_VERSION_FILE, 'w') as f:
        json.dump(versions, f, indent=2)

def check_blob_storage_changes(container_name=CONTAINER_NAME_IMAGES, prefix=MEDICAL_IMAGES_PREFIX):
    if blob_service_client is None:
        return False, None, 0
    
    try:
        container_client = blob_service_client.get_container_client(container_name)
        blobs = container_client.list_blobs(name_starts_with=prefix)
        
        image_extensions = ('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')
        image_blobs = [blob for blob in blobs if blob.name.lower().endswith(image_extensions)]
        
        if not image_blobs:
            return False, None, 0
        
        # Get latest modification time
        latest_time = max(blob.last_modified for blob in image_blobs)
        image_count = len(image_blobs)
        
        # Check against stored version
        versions = get_model_versions()
        last_trained = versions.get('ct_model', {}).get('last_trained')
        
        if last_trained:
            last_trained_dt = datetime.fromisoformat(last_trained.replace('Z', '+00:00'))
            if latest_time.replace(tzinfo=latest_time.tzinfo) > last_trained_dt.replace(tzinfo=last_trained_dt.tzinfo):
                return True, latest_time, image_count
        
        return False, latest_time, image_count
    except Exception as e:
        print(f"Error checking blob storage: {e}")
        return False, None, 0

def check_parquet_changes(parquet_blob_path=None, container_name=CONTAINER_NAME_DATA):
    if blob_service_client is None:
        return False, None
    
    if parquet_blob_path is None:
        parquet_blob_path = PARQUET_BLOB_PATH
    
    try:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=parquet_blob_path)
        blob_properties = blob_client.get_blob_properties()
        latest_time = blob_properties.last_modified
        
        # Check against stored version
        versions = get_model_versions()
        last_trained = versions.get('wellness_model', {}).get('last_trained')
        
        if last_trained:
            last_trained_dt = datetime.fromisoformat(last_trained.replace('Z', '+00:00'))
            if latest_time.replace(tzinfo=latest_time.tzinfo) > last_trained_dt.replace(tzinfo=last_trained_dt.tzinfo):
                return True, latest_time
        
        return False, latest_time
    except Exception as e:
        print(f"Error checking parquet file: {e}")
        return False, None

def trigger_model_retraining(model_type='both'):
    print("=" * 70)
    print(" " * 20 + "MODEL RETRAINING CHECK")
    print("=" * 70)
    print()
    
    versions = get_model_versions()
    retraining_needed = False
    
    # Check CT model retraining
    if model_type in ['ct', 'both']:
        print("Checking CT image data...")
        has_changes, latest_time, image_count = check_blob_storage_changes()
        if has_changes:
            print(f"    New CT images detected!")
            print(f"     Latest modification: {latest_time}")
            print(f"     Total images: {image_count}")
            print(f"     → CT model retraining recommended")
            retraining_needed = True
        else:
            print(f"  ✓ No new CT images detected")
            print(f"     Last check: {versions.get('ct_model', {}).get('last_trained', 'Never')}")
    
    print()
    
    # Check Wellness model retraining
    if model_type in ['wellness', 'both']:
        print("Checking city wellness data...")
        has_changes, latest_time = check_parquet_changes()
        if has_changes:
            print(f"    City wellness parquet file updated!")
            print(f"     Latest modification: {latest_time}")
            print(f"     → Wellness model retraining recommended")
            retraining_needed = True
        else:
            print(f"  ✓ City wellness data unchanged")
            print(f"     Last check: {versions.get('wellness_model', {}).get('last_trained', 'Never')}")
    
    print()
    print("-" * 70)
    
    if retraining_needed:
        print("RETRAINING REQUIRED:")
        print("  1. Run mlanalysis.ipynb to retrain models with new data")
        print("  2. Models will be saved to models/ directory")
        print("  3. Update model_versions.json after retraining")
        print()
        print("To update version tracking after retraining, run:")
        print("  update_model_versions()")
    else:
        print("✓ All models are up-to-date with current data")
    
    print("=" * 70)
    return retraining_needed

def update_model_versions():
    """Update model version tracking after retraining.
    Call this after running mlanalysis.ipynb to retrain models.
    """
    versions = get_model_versions()
    current_time = datetime.now().isoformat()
    
    # Update CT model version
    has_changes, latest_time, _ = check_blob_storage_changes()
    if latest_time:
        versions['ct_model'] = {
            'last_trained': current_time,
            'data_last_modified': latest_time.isoformat(),
            'model_path': os.path.join(MODEL_DIR, 'ct_risk_model.pth')
        }
    
    # Update Wellness model version
    has_changes, latest_time = check_parquet_changes()
    if latest_time:
        versions['wellness_model'] = {
            'last_trained': current_time,
            'data_last_modified': latest_time.isoformat(),
            'model_path': os.path.join(MODEL_DIR, 'wellness_model.pth'),
            'scaler_path': os.path.join(MODEL_DIR, 'wellness_scaler.pkl')
        }
    
    save_model_versions(versions)
    print("✓ Model versions updated")
    print(f"  CT Model: {versions['ct_model'].get('last_trained', 'N/A')}")
    print(f"  Wellness Model: {versions['wellness_model'].get('last_trained', 'N/A')}")

print("✓ Model retraining module ready")
print("\nTo check if retraining is needed, run: trigger_model_retraining()")
print("After retraining, update versions with: update_model_versions()")


✓ Model retraining module ready

To check if retraining is needed, run: trigger_model_retraining()
After retraining, update versions with: update_model_versions()


In [42]:
trigger_model_retraining()

                    MODEL RETRAINING CHECK

Checking CT image data...
  ✓ No new CT images detected
     Last check: None

Checking city wellness data...
  ✓ City wellness data unchanged
     Last check: None

----------------------------------------------------------------------
✓ All models are up-to-date with current data


False

## Dev testing cell

In [ ]:

customer_id = 1
city = None 
ct_image_blob = None   
base_premium = 200  

try:
    print("=" * 60)
    print("INSURANCE QUOTE GENERATION PIPELINE")
    print("=" * 60)
    
    print(f"\n[STEP 1] Processing customer {customer_id}...")
    customer = get_customer(customer_id)
    if not customer:
        raise ValueError(f"Customer {customer_id} not found in database")
    
    print(f"  ✓ Customer found: {customer.get('FirstName', 'N/A')} {customer.get('LastName', 'N/A')}")
    
    print(f"\n[STEP 2] Computing ML scores...")
    scores = ensure_scores(
        customer_id=customer_id, 
        city=city, 
        ct_image_blob=ct_image_blob,
        from_blob=True  # Load from Azure Blob Storage
    )
    print(f"  ✓ CT Risk Score: {scores['CT_RiskScore']:.4f}")
    print(f"  ✓ Wellness Score: {scores['WellnessScore']:.4f}")
    
    print(f"\n[STEP 3] Calculating premium...")
    premium = compute_premium(base_premium, scores["CT_RiskScore"], scores["WellnessScore"])
    print(f"  Base Premium: ${base_premium:.2f}")
    print(f"  Final Premium: ${premium:.2f}")
    
    print(f"\n[STEP 4] Creating contract...")
    contract_id = insert_contract(customer_id)
    print(f"  ✓ Contract created with ID: {contract_id}")
    
    print(f"\n[STEP 5] Creating contract benefit...")
    contract_benefit_id = None
    try:
        contract_benefit_id = insert_contract_benefit(contract_id, benefit_type="Health", coverage_amount=50000.00)
        print(f"  ✓ Contract benefit created with ID: {contract_benefit_id}")
        
        print(f"\n[STEP 6] Inserting premium...")
        insert_contract_premium(contract_benefit_id, premium)
        print(f"  ✓ Premium inserted: ${premium:.2f}")
    except ValueError as e:
        print(f" {e}")
        print(f" Skipping ContractBenefit and ContractPremium insertion.")
        print(f"  Contract {contract_id} was created, but premium cannot be inserted without ContractBenefit.")
    
    print(f"\n[STEP 7] Refreshing materialized views...")
    refresh_materialized_views()
    print(f"  ✓ Materialized views refreshed")
    
    print("\n" + "=" * 60)
    print("QUOTE SUMMARY")
    print("=" * 60)
    print(f"Customer ID:        {customer_id}")
    print(f"Customer Name:     {customer.get('FirstName', 'N/A')} {customer.get('LastName', 'N/A')}")
    print(f"City:              {city or customer.get('City', 'N/A')}")
    print(f"CT Image Blob:     {ct_image_blob or 'Retrieved from database'}")
    print(f"CT Risk Score:     {scores['CT_RiskScore']:.4f}")
    print(f"Wellness Score:    {scores['WellnessScore']:.4f}")
    print(f"Base Premium:      ${base_premium:.2f}")
    print(f"Final Premium:     ${premium:.2f}")
    print(f"Contract ID:       {contract_id}")
    if contract_benefit_id:
        print(f"Contract Benefit:  {contract_benefit_id}")
    else:
        print(f"Contract Benefit:  Not created")
    print("=" * 60)
    print("✓ Quote generated successfully!")
    print("=" * 60)
    
except Exception as e:
    print(f"\n !! ERROR: {e}")
    import traceback
    traceback.print_exc()


INSURANCE QUOTE GENERATION PIPELINE

[STEP 1] Processing customer 1...
  ✓ Customer found: Ava Johnson

[STEP 2] Computing ML scores...
  ✓ CT Risk Score: 0.7003
  ✓ Wellness Score: 0.5549

[STEP 3] Calculating premium...
  Base Premium: $200.00
  Final Premium: $309.43

[STEP 4] Creating contract...
  ✓ Contract created with ID: 26

[STEP 5] Creating contract benefit...
  ✓ Contract benefit created with ID: 24

[STEP 6] Inserting premium...
  ✓ Premium inserted: $309.43

[STEP 7] Refreshing materialized views...
  ✓ Materialized views refreshed

QUOTE SUMMARY
Customer ID:        1
Customer Name:     Ava Johnson
City:              New York
CT Image Blob:     Retrieved from database
CT Risk Score:     0.7003
Wellness Score:    0.5549
Base Premium:      $200.00
Final Premium:     $309.43
Contract ID:       26
Contract Benefit:  24
✓ Quote generated successfully!
